#1. Install Dependencies
First install the libraries needed to execute recipes, this only needs to be done once, then click play.


In [ ]:
!pip install git+https://github.com/google/starthinker


#2. Get Cloud Project ID
To run this recipe [requires a Google Cloud Project](https://github.com/google/starthinker/blob/master/tutorials/cloud_project.md), this only needs to be done once, then click play.


In [ ]:
CLOUD_PROJECT = 'PASTE PROJECT ID HERE'

print("Cloud Project Set To: %s" % CLOUD_PROJECT)


#3. Get Client Credentials
To read and write to various endpoints requires [downloading client credentials](https://github.com/google/starthinker/blob/master/tutorials/cloud_client_installed.md), this only needs to be done once, then click play.


In [ ]:
CLIENT_CREDENTIALS = 'PASTE CREDENTIALS HERE'

print("Client Credentials Set To: %s" % CLIENT_CREDENTIALS)


#4. Enter DV360 Segmentology Parameters
DV360 funnel analysis using Census data.
 1. Wait for <b>BigQuery->->->Census_Join</b> to be created.
 1. Join the <a hre='https://groups.google.com/d/forum/starthinker-assets' target='_blank'>StarThinker Assets Group</a> to access the following assets
 1. Copy <a href='https://datastudio.google.com/c/u/0/reporting/3673497b-f36f-4448-8fb9-3e05ea51842f/' target='_blank'>DV360 Segmentology Sample</a>. Leave the Data Source as is, you will change it in the next step.
 1. Click Edit Connection, and change to <b>BigQuery->->->Census_Join</b>.
 1. Or give these intructions to the client.
Modify the values below for your use case, can be done multiple times, then click play.


In [ ]:
FIELDS = {
  'auth_read': 'user',  # Credentials used for reading data.
  'recipe_timezone': 'America/Los_Angeles',  # Timezone for report dates.
  'recipe_project': '',  # Project ID hosting dataset.
  'auth_write': 'service',  # Authorization used for writing data.
  'recipe_name': '',  # Name of report, not needed if ID used.
  'recipe_slug': '',  # Name of Google BigQuery dataset to create.
  'partners': [],  # DV360 partner id.
  'advertisers': [],  # Comma delimited list of DV360 advertiser ids.
}

print("Parameters Set To: %s" % FIELDS)


#5. Execute DV360 Segmentology
This does NOT need to be modified unless you are changing the recipe, click play.


In [ ]:
from starthinker.util.project import project
from starthinker.script.parse import json_set_fields

USER_CREDENTIALS = '/content/user.json'

TASKS = [
  {
    'dataset': {
      'description': 'Create a dataset for bigquery tables.',
      'hour': [
        4
      ],
      'auth': 'user',
      'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be created in BigQuery.'}}
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'function': 'Pearson Significance Test',
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}}
      }
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'filters': {
          'FILTER_PARTNER': {
            'values': {'field': {'name': 'partners','kind': 'integer_list','order': 5,'default': [],'description': 'DV360 partner id.'}}
          },
          'FILTER_ADVERTISER': {
            'values': {'field': {'name': 'advertisers','kind': 'integer_list','order': 6,'default': [],'description': 'Comma delimited list of DV360 advertiser ids.'}}
          }
        },
        'body': {
          'timezoneCode': {'field': {'name': 'recipe_timezone','kind': 'timezone','description': 'Timezone for report dates.','default': 'America/Los_Angeles'}},
          'metadata': {
            'title': {'field': {'name': 'recipe_name','kind': 'string','order': 3,'prefix': 'Segmentology ','default': '','description': 'Name of report, not needed if ID used.'}},
            'dataRange': 'LAST_30_DAYS',
            'format': 'CSV'
          },
          'params': {
            'type': 'TYPE_CROSS_PARTNER',
            'groupBys': [
              'FILTER_PARTNER',
              'FILTER_PARTNER_NAME',
              'FILTER_ADVERTISER',
              'FILTER_ADVERTISER_NAME',
              'FILTER_MEDIA_PLAN',
              'FILTER_MEDIA_PLAN_NAME',
              'FILTER_ZIP_POSTAL_CODE'
            ],
            'metrics': [
              'METRIC_BILLABLE_IMPRESSIONS',
              'METRIC_CLICKS',
              'METRIC_TOTAL_CONVERSIONS'
            ]
          },
          'schedule': {
            'frequency': 'WEEKLY'
          }
        }
      }
    }
  },
  {
    'dbm': {
      'auth': 'user',
      'report': {
        'name': {'field': {'name': 'recipe_name','kind': 'string','order': 3,'prefix': 'Segmentology ','default': '','description': 'Name of report, not needed if ID used.'}}
      },
      'out': {
        'bigquery': {
          'auth': 'user',
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
          'table': 'DV360_KPI',
          'header': True,
          'schema': [
            {
              'name': 'Partner_Id',
              'type': 'INTEGER',
              'mode': 'REQUIRED'
            },
            {
              'name': 'Partner',
              'type': 'STRING',
              'mode': 'REQUIRED'
            },
            {
              'name': 'Advertiser_Id',
              'type': 'INTEGER',
              'mode': 'REQUIRED'
            },
            {
              'name': 'Advertiser',
              'type': 'STRING',
              'mode': 'REQUIRED'
            },
            {
              'name': 'Campaign_Id',
              'type': 'INTEGER',
              'mode': 'REQUIRED'
            },
            {
              'name': 'Campaign',
              'type': 'STRING',
              'mode': 'REQUIRED'
            },
            {
              'name': 'Zip',
              'type': 'STRING',
              'mode': 'NULLABLE'
            },
            {
              'name': 'Impressions',
              'type': 'FLOAT',
              'mode': 'NULLABLE'
            },
            {
              'name': 'Clicks',
              'type': 'FLOAT',
              'mode': 'NULLABLE'
            },
            {
              'name': 'Conversions',
              'type': 'FLOAT',
              'mode': 'NULLABLE'
            }
          ]
        }
      }
    }
  },
  {
    'bigquery': {
      'auth': 'user',
      'from': {
        'query': 'SELECT            Partner_Id,            Partner,            Advertiser_Id,            Advertiser,            Campaign_Id,            Campaign,            Zip,            SAFE_DIVIDE(Impressions, SUM(Impressions) OVER(PARTITION BY Advertiser_Id)) AS Impression_Percent,            SAFE_DIVIDE(Clicks, Impressions) AS Click_Percent,            SAFE_DIVIDE(Conversions, Impressions) AS Conversion_Percent,            Impressions AS Impressions          FROM            `{project}.{dataset}.DV360_KPI`;        ',
        'parameters': {
          'project': {'field': {'name': 'recipe_project','kind': 'string','description': 'Project ID hosting dataset.'}},
          'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be created in BigQuery.'}}
        },
        'legacy': False
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','description': 'Place where tables will be written in BigQuery.'}},
        'view': 'DV360_KPI_Normalized'
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'normalize': {
        'census_geography': 'zip_codes',
        'census_year': '2018',
        'census_span': '5yr'
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'type': 'view'
      }
    }
  },
  {
    'census': {
      'auth': 'user',
      'correlate': {
        'join': 'Zip',
        'pass': [
          'Partner_Id',
          'Partner',
          'Advertiser_Id',
          'Advertiser',
          'Campaign_Id',
          'Campaign'
        ],
        'sum': [
          'Impressions'
        ],
        'correlate': [
          'Impression_Percent',
          'Click_Percent',
          'Conversion_Percent'
        ],
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'table': 'DV360_KPI_Normalized',
        'significance': 80
      },
      'to': {
        'dataset': {'field': {'name': 'recipe_slug','kind': 'string','order': 4,'default': '','description': 'Name of Google BigQuery dataset to create.'}},
        'type': 'view'
      }
    }
  }
]

json_set_fields(TASKS, FIELDS)

project.initialize(_recipe={ 'tasks':TASKS }, _project=CLOUD_PROJECT, _user=USER_CREDENTIALS, _client=CLIENT_CREDENTIALS, _verbose=True, _force=True)
project.execute(_force=True)
